# Battle of Neighborhoods

## Introduction

its main idea to help the stake holders to achieve the desired results, so as to help them spend their time and money in some other productive means rather keep them trapped in an infinite loop of extensive search engines.
 
 help the stakeholders take a better decision on choosing the best neighborhood out of many neighborhoods to build/buy their houses in Seattle city based on the distribution of various facilities in and around that neighborhood. As an example, this project would compare 2 randomly picked neighborhoods and analyses the top 5 most common venues in each of those two neighborhoods based on the number of visits by people in each of those places.

## Data

Four-square API as its prime data gathering source as it has a database of more than 105 million places, especially their places API which provides the ability to perform location search, location sharing and details about a business. Photos, tips and reviews jolted by Foursquare users can also be used in many productive ways to add value to the results.

## Methodology

HTTP requests would be made to this Foursquare API server using zip codes of the Seattle city neighborhoods to pull the location information (Latitude and Longitude).

Foursquare API search feature would be enabled to collect the nearby places of the neighborhoods. The number of places per neighborhood parameter would reasonably be set to 50 and the radius parameter would be set to 100.

Folium- Python visualization library would be used to visualize the neighborhoods cluster distribution of Seattle city over an interactive leaflet map.

Unsupervised machine learning algorithm K-mean clustering would be applied to form the clusters of different categories of places residing in and around the neighborhoods.

### Import Libraries

In [1]:
import requests # library to handle requests
import json # library to handle JSON files
import xml
import pandas as pd #library data manipulation and analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup #Python Web scraping library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

### Collecting Neighborhoods and their Postalcodes

In [39]:
# from url = 'http://seattlearea.com/zip-codes/'


neighborhood=['Federal Way', 'Bellevue', 'Kirkland', 'Lynnwood', 'Mercer Island',
              'Redmond', 'Renton', 'Seattle', 'Downtown', 'Capital Hill', 
              'Greenwood','Freemont', 'Greenlake', 'International District', 
              'Pioneer Square', 'University District', 'Laurelhurst', 'Ballard',
              'South Lake Union', 'Queen Anne', 'Bainbridge Island', 'Madrona',
              'West Seattle', 'Alki Beach', 'Columbia City', 'Belltown',
              'Northgate', 'Mount Baker', 'Magnolia']
zipcode=[98003, 98005, 98033, 98037, 98040, 98052, 98055, 98101, 98101, 98102,
         98103, 98103, 98103, 98104, 98104, 98105, 98105, 98107, 98109, 98109,
         98110, 98110, 98116, 98116, 98118, 98121, 98125, 98144, 98199]

print(len(neighborhood))
print(len(zipcode))


29
29


In [40]:
sea_df=pd.DataFrame({'PostalCode':zipcode,
                 'Neighborhood':neighborhood})
sea_df.head()

,Neighborhood,PostalCode
0,Federal Way,98003
1,Bellevue,98005
2,Kirkland,98033
3,Lynnwood,98037
4,Mercer Island,98040


### Obtaining latitude and longitude

In [43]:
latitude=[] #List to collect the latitudes
longitude=[] #List to collect the longitudes

for i in sea_df['PostalCode']: #Iterating through Postalcodes to collect the locations data
    try:
        
        url ="https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}".format(API_key,i)
        response = requests.get(url).json() # get response
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        latitude.append(geographical_data['lat'])
        longitude.append(geographical_data['lng'])
    except:
        pass
sea_df['Latitude']=latitude #Adding a column in the main dataframe for Latitude  

sea_df['Longitude']=longitude #Adding a column in the main dataframe for Longitude 
sea_df.head()

,Neighborhood,PostalCode,Latitude,Longitude
0,Federal Way,98003,47.316504,-122.322397
1,Bellevue,98005,47.615044,-122.171758
2,Kirkland,98033,47.668830,-122.192387
3,Lynnwood,98037,47.841952,-122.288181
4,Mercer Island,98040,47.582423,-122.233123


In [44]:
address = 'Seattle,Washington'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude


### create map of Seatle

In [47]:
map_sea = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# add markers to map
for lat, lng, nei in zip(sea_df['Latitude'], 
                         sea_df['Longitude'],
                         sea_df['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sea)

label = '{}'.format(address)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [latitude_x, longitude_y],
    radius=7,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_sea)  

map_sea

### obtaining info from FOURSQUARE

In [89]:
address = 'Seattle,Washington'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_n1 = location.latitude
longitude_n1 = location.longitude

radius = 1000 
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    Client_ID, 
    Client_Secret, 
    VERSION, 
    latitude_n1, 
   longitude_n1, 
    radius, 
   LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d06cff1d9a6e6003309d5ed'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4db71ec1a86ed8d46c6e179c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d110941735',
         'name': 'Italian Restaurant',
         'pluralName': 'Italian Restaurants',
         'primary': True,
         'shortName': 'Italian'}],
       'id': '4db71ec1a86ed8d46c6e179c',
       'location': {'address': '217 James St',
        'cc': 'US',
        'city': 'Seattle',
        'country': 'United States',
        'crossStreet': 'btwn 2nd & 3rd Ave',
        'distance': 203,
        'formattedAddress': ['217 James St (btwn 2nd & 3rd Ave)',
         'Seattle, WA 98104',
         'United States'

In [91]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
filtered_columns = ['venue.name', 'venue.categories', 
                    'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Il Corvo,Italian Restaurant,47.602522,-122.331952
1,Biscuit B*tch,Breakfast Spot,47.603237,-122.332010
2,Columbia Tower Club,Social Club,47.604507,-122.330484
3,Juicy Cafe,Café,47.604329,-122.330958
4,Top Pot Doughnuts,Donut Shop,47.604023,-122.332499


In [92]:

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            Client_ID, 
            Client_Secret, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


sea_venues = getNearbyVenues(names=sea_df['Neighborhood'],
                                   latitudes=sea_df['Latitude'],
                                   longitudes=sea_df['Longitude']
                                  )

sea_venues.groupby('Neighborhood').count()

Federal Way
Bellevue
Kirkland
Lynnwood
Mercer Island
Redmond
Renton
Seattle
Downtown
Capital Hill
Greenwood
Freemont
Greenlake
International District
Pioneer Square
University District
Laurelhurst
Ballard
South Lake Union
Queen Anne
Bainbridge Island
Madrona
West Seattle
Alki Beach
Columbia City
Belltown
Northgate
Mount Baker
Magnolia


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alki Beach,50,50,50,50,50,50
Bainbridge Island,1,1,1,1,1,1
Ballard,50,50,50,50,50,50
Bellevue,36,36,36,36,36,36
Belltown,50,50,50,50,50,50
Capital Hill,50,50,50,50,50,50
Columbia City,21,21,21,21,21,21
Downtown,50,50,50,50,50,50
Federal Way,50,50,50,50,50,50


### Verifying the venues

In [93]:

# one hot encoding
sea_onehot = pd.get_dummies(sea_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sea_onehot['Neighborhood'] = sea_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sea_onehot.columns[-1]] + list(sea_onehot.columns[:-1])
sea_onehot = sea_onehot[fixed_columns]
sea_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Bike Trail,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Herbs & Spices Store,Historic Site,History Museum,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Light Rail Station,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Sake Bar,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Social Club,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Winery
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Federal Way,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Federal Way,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Federal Way,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

### Obtaining the results for each neighborhood


In [94]:
sea_grouped = sea_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 3
for hood in sea_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =sea_grouped[sea_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alki Beach----
            venue  freq
0     Coffee Shop  0.08
1  Ice Cream Shop  0.08
2            Park  0.08


----Bainbridge Island----
         venue  freq
0   Skate Park   1.0
1  Yoga Studio   0.0
2    Multiplex   0.0


----Ballard----
            venue  freq
0         Brewery  0.08
1  Ice Cream Shop  0.06
2     Gaming Cafe  0.04


----Bellevue----
                    venue  freq
0     Rental Car Location  0.06
1            Carpet Store  0.06
2  Furniture / Home Store  0.06


----Belltown----
              venue  freq
0               Bar  0.08
1    Breakfast Spot  0.06
2  Sushi Restaurant  0.06


----Capital Hill----
            venue  freq
0     Coffee Shop  0.10
1  Sandwich Place  0.06
2          Garden  0.06


----Columbia City----
                venue  freq
0  Mexican Restaurant  0.19
1                Park  0.10
2   Convenience Store  0.05


----Downtown----
                     venue  freq
0              Coffee Shop  0.10
1      American Restaurant  0.06
2  New American 

### Analizing the information

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sea_grouped['Neighborhood']

for ind in np.arange(sea_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sea_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Beach,Ice Cream Shop,Coffee Shop,Park,Italian Restaurant,Beach,Thai Restaurant,Seafood Restaurant,Brewery,Mexican Restaurant,History Museum
1,Bainbridge Island,Skate Park,Winery,Dog Run,Fast Food Restaurant,Farmers Market,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
2,Ballard,Brewery,Ice Cream Shop,Pet Store,Coffee Shop,Cocktail Bar,Mexican Restaurant,Gaming Cafe,Rock Club,Farmers Market,Boutique
3,Bellevue,Park,Coffee Shop,Carpet Store,Rental Car Location,Furniture / Home Store,Martial Arts Dojo,Residential Building (Apartment / Condo),Botanical Garden,Seafood Restaurant,Pharmacy
4,Belltown,Bar,Breakfast Spot,Sushi Restaurant,Gastropub,Beer Bar,Steakhouse,Bakery,Cocktail Bar,Thai Restaurant,Sculpture Garden


### Cloustering

In [97]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sea_grouped['Neighborhood']

for ind in np.arange(sea_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sea_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Beach,Ice Cream Shop,Coffee Shop,Park,Italian Restaurant,Beach,Thai Restaurant,Seafood Restaurant,Brewery,Mexican Restaurant,History Museum
1,Bainbridge Island,Skate Park,Winery,Dog Run,Fast Food Restaurant,Farmers Market,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
2,Ballard,Brewery,Ice Cream Shop,Pet Store,Coffee Shop,Cocktail Bar,Mexican Restaurant,Gaming Cafe,Rock Club,Farmers Market,Boutique
3,Bellevue,Park,Coffee Shop,Carpet Store,Rental Car Location,Furniture / Home Store,Martial Arts Dojo,Residential Building (Apartment / Condo),Botanical Garden,Seafood Restaurant,Pharmacy
4,Belltown,Bar,Breakfast Spot,Sushi Restaurant,Gastropub,Beer Bar,Steakhouse,Bakery,Cocktail Bar,Thai Restaurant,Sculpture Garden


In [104]:
# Using k-means to cluster the neighborhood into 10 clusters.
kclusters = 10
sea_grouped_clustering = sea_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sea_grouped_clustering)

sea_merged = sea_df
sea_merged['Cluster Labels'] = kmeans.labels_

sea_merged = sea_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
sea_merged # check the last columns!

,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Federal Way,98003,47.316504,-122.322397,4,Mexican Restaurant,Coffee Shop,Pet Store,Grocery Store,Miscellaneous Shop,Japanese Restaurant,Korean Restaurant,Thai Restaurant,Chinese Restaurant,Café
1,Bellevue,98005,47.615044,-122.171758,1,Park,Coffee Shop,Carpet Store,Rental Car Location,Furniture / Home Store,Martial Arts Dojo,Residential Building (Apartment / Condo),Botanical Garden,Seafood Restaurant,Pharmacy
2,Kirkland,98033,47.668830,-122.192387,4,Asian Restaurant,Sandwich Place,Coffee Shop,Brewery,Pharmacy,Music Venue,Food Truck,Frozen Yogurt Shop,Gas Station,Organic Grocery
3,Lynnwood,98037,47.841952,-122.288181,8,Pizza Place,Fast Food Restaurant,Coffee Shop,Spa,Food,Korean Restaurant,Convenience Store,Park,Pharmacy,Sandwich Place
4,Mercer Island,98040,47.582423,-122.233123,2,Coffee Shop,Pizza Place,Park,Pharmacy,Gym / Fitness Center,Sandwich Place,Thai Restaurant,New American Restaurant,Shipping Store,Salon / Barbershop
5,Redmond,98052,47.670119,-122.118237,8,Bakery,Gym / Fitness Center,Theater,Mexican Restaurant,Burger Joint,Dumpling Restaurant,Cycle Studio,Seafood Restaurant,Karaoke Bar,Korean Restaurant
6,Renton,98055,47.462337,-122.205506,6,Residential Building (Apartment / Condo),Fast Food Restaurant,Asian Restaurant,Business Service,Bar,Café,Bakery,Optical Shop,Bike Trail,Deli / Bodega
7,Seattle,98101,47.608492,-122.336407,5,Coffee Shop,New American Restaurant,American Restaurant,Hotel,Concert Hall,Spa,Gourmet Shop,Market,Italian Restaurant,Filipino Restaurant
8,Downtown,98101,47.608492,-122.336407,8,Coffee Shop,New American Restaurant,American Restaurant,Hotel,Concert Hall,Spa,Gourmet Shop,Market,Italian Restaurant,Filipino Restaurant
9,Capital Hill,98102,47.633822,-122.321545,2,Coffee Shop,Sandwich Place,Garden,Italian Restaurant,Restaurant,Scenic Lookout,Bar,Pizza Place,Men's Store,Boat or Ferry


### Creating map

In [105]:
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(sea_merged['Latitude'], 
                                   sea_merged['Longitude'],
                                   sea_merged['Neighborhood'], 
                                   sea_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']


In [107]:
df1=sea_merged.loc[sea_merged['Cluster Labels'] == 0,
                       sea_merged.columns[[0] + list(range(5, sea_merged.shape[1]))]]

df2=sea_merged.loc[sea_merged['Cluster Labels'] == 1,
                   sea_merged.columns[[0] + list(range(5, sea_merged.shape[1]))]]

df3=sea_merged.loc[sea_merged['Cluster Labels'] == 2,
                   sea_merged.columns[[0] + list(range(5, sea_merged.shape[1]))]]

df4=sea_merged.loc[sea_merged['Cluster Labels'] == 3,
                   sea_merged.columns[[0] + list(range(5, sea_merged.shape[1]))]]

df5=sea_merged.loc[sea_merged['Cluster Labels'] == 4,
                   sea_merged.columns[[0] + list(range(5, sea_merged.shape[1]))]]

clusters=pd.DataFrame({"Cluster1":df1["Neighborhood"],
                      "Cluster2":df2["Neighborhood"],
                      "Cluster3":df3["Neighborhood"],
                      "Cluster4":df4["Neighborhood"],
                      "Cluster5":df5["Neighborhood"],})

clusters = clusters.replace(np.nan, '', regex=True)

clusters

,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5
0,,,,,Federal Way
1,,Bellevue,,,
2,,,,,Kirkland
4,,,Mercer Island,,
9,,,Capital Hill,,
10,,,Greenwood,,
11,,,Freemont,,
12,Greenlake,,,,
14,,,,Pioneer Square,
16,,Laurelhurst,,,


### Conclussion

With this tool you can select the best neighborhood according to your preferences and hobbies, in this way you can obtain using cloustering techniques the neighborhoods similar to your preferences.